In [3]:
import pandas as pd
import numpy as np
import torch
from  torch import nn, optim
import torch.nn.functional as F 
import matplotlib.pyplot as plt
from string import punctuation
from sklearn.metrics import accuracy_score


In [11]:
data_init = pd.read_csv("amazon_cells_labelled.txt", sep = "\t", header = None)

In [9]:
for i in punctuation:
    data[0] = data[0].str.replace(i, "")

C:\Users\devan\AppData\Local\Temp\ipykernel_16772\277504227.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[0] = data[0].str.replace(i, "")


In [27]:
reviews = data.iloc[:,0].str.lower()
sentiment = data[1].values

In [40]:
words = ' '.join(reviews)
words = words.split()

In [44]:
indexer = {word: index for (index, word) in enumerate(list(set(words)))}

KeyError: 's'

In [45]:
indexed_reviews = []
for review in reviews:
    indexed_reviews.append([indexer[word] for word in review.split()])

In [ ]:
  '''def forward(self, x):
        out = self.embedding(x)
        out, _ = self.lstm(out)
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.output(out)
        out = out[-1,0]
        out = torch.sigmoid(out).unsqueeze(0)
        return out'''

In [115]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, n_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_size, n_layers, batch_first = True)
        self.output = nn.Linear(hidden_size, 1)
    def forward(self,  x):
        out = self.embedding(x)
#         print(out.shape)
        out, state = self.lstm(out)
#         print(out.shape)
        out = out.contiguous().view(-1,  self.hidden_size)
        print(out.shape)
        out = self.output(out)
        print(out.shape)
        out = torch.sigmoid(out).unsqueeze(0)
        print(out.shape)
        out = out[-1, -1]
        print(out.shape)
        out = torch.sigmoid(out).unsqueeze(0)
        print(out.shape)
        print("done")
        return out

In [116]:
model = LSTM(len(indexer), 64, 128, 3)
model

LSTM(
  (embedding): Embedding(1905, 64)
  (lstm): LSTM(64, 128, num_layers=3, batch_first=True)
  (output): Linear(in_features=128, out_features=1, bias=True)
)

In [117]:
loss_function = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10

In [120]:
losses = []

acc = []

for e in range(1, epochs+1):

    single_loss = []

    preds = []

    targets = []

    for i, r in enumerate(indexed_reviews):

        if len(r) <= 1:

            continue

        x = torch.Tensor([r]).long()

        y = torch.Tensor([sentiment[i]])

        pred = model(x)

        loss = loss_function(pred, y)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        final_pred = np.round(pred.detach().numpy())

        preds.append(final_pred)

        targets.append(y.item())

        single_loss.append(loss.item())

    losses.append(np.mean(single_loss))

    accuracy = 0

    acc.append(accuracy)

    if e%1 == 0:

        print("Epoch: ", e, "... Loss function: ", losses[-1],"... Accuracy: ", acc[-1])

Epoch:  1 ... Loss function:  0.6420136964645694 ... Accuracy:  0
Epoch:  2 ... Loss function:  0.47061644874886055 ... Accuracy:  0
Epoch:  3 ... Loss function:  0.32483265519594695 ... Accuracy:  0
Epoch:  4 ... Loss function:  0.23858983721584082 ... Accuracy:  0
Epoch:  5 ... Loss function:  0.14235232906756637 ... Accuracy:  0
Epoch:  6 ... Loss function:  0.08337759355132766 ... Accuracy:  0
Epoch:  7 ... Loss function:  0.06005842961165754 ... Accuracy:  0
Epoch:  8 ... Loss function:  0.059374022819907585 ... Accuracy:  0
Epoch:  9 ... Loss function:  0.0316375541807655 ... Accuracy:  0
Epoch:  10 ... Loss function:  0.022348778906553072 ... Accuracy:  0


In [142]:
review = " "
pred =  model(torch.tensor([indexer[word] for word in review.split()]))
print(pred.item())

KeyError: 'sarcasm'